# Questão 02)

## Mínimos quadrados

In [54]:
import pandas as pd
import numpy as np

import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split

In [55]:
# Read xlsx file
ws = pd.read_excel("Real_estate_valuation_dataset.xlsx", engine='openpyxl')
# Drop useless column
ws = ws.drop('No', axis=1)
# Convert in numpy ndarray
data = np.array(ws)

In [56]:
ws.head()

,X1 transaction date,X2 house age,X3 distance to the nearest MRT station,X4 number of convenience stores,X5 latitude,X6 longitude,Y house price of unit area
0,2012.916667,32.0,84.87882,10,24.98298,121.54024,37.9
1,2012.916667,19.5,306.59470,9,24.98034,121.53951,42.2
2,2013.583333,13.3,561.98450,5,24.98746,121.54391,47.3
3,2013.500000,13.3,561.98450,5,24.98746,121.54391,54.8
4,2012.833333,5.0,390.56840,5,24.97937,121.54245,43.1


In [57]:
X = data[:, :-1]
Y = data[:, -1]

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

print("x_train:", x_train.shape)
print("y_train:", y_train.shape)
print("x_test:", x_test.shape)
print("y_test:", y_test.shape)

x_train: (331, 6)
y_train: (331,)
x_test: (83, 6)
y_test: (83,)


In [58]:
def ridge_regression(X, y, alpha):
    # Adiciona uma coluna de 1s para representar o termo de viés
    X = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)

    # Calcula os coeficientes usando a fórmula dos mínimos quadrados com regularização de Ridge
    theta = np.linalg.inv(X.T @ X + alpha * np.eye(X.shape[1])) @ X.T @ y

    return theta

def calculate_error(X, y, theta):
    # Adiciona uma coluna de 1s para representar o termo de viés
    X = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)

    # Realiza as predições
    predictions = X @ theta

    # Converte as predições em rótulos
    predicted_labels = np.argmax(predictions, axis=1)

    # Calcula o erro (taxa de erro)
    error = np.mean(predicted_labels != y)

    return error


In [59]:

# Define o valor de regularização (alpha)
alpha = 0.01

# Realiza a regressão usando mínimos quadrados com regularização de Ridge
theta = ridge_regression(x_train, y_train, alpha)

# Adiciona uma coluna de 1s aos dados de teste
X_test = np.concatenate((np.ones((x_test.shape[0], 1)), x_test), axis=1)

# Realiza as predições
predictions = X_test @ theta

# Calcular o erro médio quadrado (MSE) nas previsões
mse = mean_squared_error(y_test, predictions)
print("Erro médio quadrado (MSE):", mse)
mae = mean_absolute_error(y_test, predictions)
print("Erro médio absoluto (MAE):", mae)

Erro médio quadrado (MSE): 54.60475048894781
Erro médio absoluto (MAE): 5.326757449309436


## ELM

## MLP

In [121]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')


# Read xlsx file
ws = pd.read_excel("Real_estate_valuation_dataset.xlsx", engine='openpyxl')
# Drop useless column
ws = ws.drop('No', axis=1)
# Convert to numpy ndarray
data = np.array(ws)

X = data[:, :-1]
Y = data[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

def init(x, y):
    layer = np.random.uniform(-1, 1., size=(x, y)) / np.sqrt(x * y)
    return layer.astype(np.float32)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def d_sigmoid(x):
    return x * (1 - x)

def forward_backward_pass(x, y):
    x_l1 = x.dot(l1)
    x_sigmoid = sigmoid(x_l1)

    x_l2 = x_sigmoid.dot(l2)
    out = x_l2
    
    error = 2 * (out - y) / out.shape[0]
    update_l2 = x_sigmoid.T @ error
    
    error = (error @ l2.T) * d_sigmoid(x_sigmoid)
    update_l1 = x.T @ error
    
    return out, update_l1, update_l2

epochs = 1000
lr = 0.001
batch = 32

np.random.seed(42)
l1 = init(X_train.shape[1], 24)
l2 = init(24, 1)

losses = []

y = y.reshape((-1, 1))

for i in range(epochs):
    sample = np.random.randint(0, X_train.shape[0], size=(batch))
    x = X_train[sample]
    y = y_train[sample].reshape((-1, 1))

    out, update_l1, update_l2 = forward_backward_pass(x, y)

    loss = mean_squared_error(y, out)  # Calculate MSE
    losses.append(loss)

    l1 -= lr * update_l1
    l2 -= lr * update_l2

    if i % 100 == 0:
        print(f'Epoch {i}, Loss: {loss}')


X_test = X_test.reshape((-1, X_train.shape[1]))
y_test = y_test.reshape((-1, 1))

# Teste
def test(x, y, l1, l2):
    x_l1 = x.dot(l1)
    x_sigmoid = sigmoid(x_l1)

    x_l2 = x_sigmoid.dot(l2)
    out = x_l2
    
    return out

# Calculando as previsões para o conjunto de testes
y_pred_test = test(X_test, y_test, l1, l2)

# Calculando o MSE para o conjunto de testes
mse_test = mean_squared_error(y_test, y_pred_test)
print("Test MSE:", mse_test)

# Calculando o MAE para o conjunto de testes
mae_test = mean_absolute_error(y_test, y_pred_test)
print("Test MAE:", mae_test)

Epoch 0, Loss: 1744.3532001788842
Epoch 100, Loss: 166.20439441686747
Epoch 200, Loss: 170.97232164491703
Epoch 300, Loss: 88.55545742796599
Epoch 400, Loss: 230.98242021818038
Epoch 500, Loss: 144.75508647085684
Epoch 600, Loss: 154.17392710953555
Epoch 700, Loss: 343.88520207327196
Epoch 800, Loss: 116.81917441293152
Epoch 900, Loss: 338.81722664754415
Test MSE: 171.54173666490547
Test MAE: 10.89998090791787
